<h3><center>Introduction to building a quant pricing library</center></h3>
<h1><center>Option Pricing by Partial Differential Equation using OOP</center></h1>
<h3><center>Abdulwahab, Animoku</center></h3>
<h4><center>18 October, 2023</center></h4>



## Option Pricing by PDE

<b>Goal of option pricing via PDE:<b>

* Use the Black Scholes PDE to price European Contract
* Use  different discretization scheme to achieve better results
* Compare the solution to the Analytical Pricer
* Price American Option with PDE method


<b>Extension<b>
* Calculate Greeks based on the different Schemes
* Extend PDE pricing scheme to Forward<b>

# Steps to achieve our Objectives
* Create Market Model
* Create European Contract using market inputs (underlying, Call/Put, Long/Short, strike, expiry)
* Create PDEParams using $dS, dt, S_{min_{multiplier}}, S_{max_{multiplier}} $ and method (implicit, explicit, CrankNicolson) as inputs
* Create EuropeanPDEPricer using the contract, model and PDEParams
* Calculate FV
* Visualize the Call and Put prices in 2D

In [ ]:
import matplotlib.pyplot as plt

to_plot = [ {'data': [[0]*6, list(range(6))],
             'color':'r',
             'title':'Our interests'},
           {'data': [list(range(6))+[5]*6+list(range(6)), [0]*6+list(range(6))+[5]*6],
            'color' : 'g',
            'title' : 'Boundary and initial conditions'
           },] + [
           {'data': [[i]*6, list(range(6))],
            'color' : 'g',
            'title' : f'Step {5-i}'
           } for i in range(4,-1,-1)]
           

def my_illustration_plot(to_plot):
    fig,ax = plt.subplots(1)
    for x in to_plot:
        plt.scatter(*x['data'], s=300, c=x['color'])
        plt.title(x['title'])
#         plt.axis('on')
        plt.xlim(-1,6)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.xlabel("Time")
    plt.ylabel("Stock")
    plt.show()
    

In [ ]:
my_illustration_plot(to_plot[:6])

## Call and Put Option Pricing for European Options via PDE

Recall that the value $V$ of a Europoean option at time $t$, written on an underlying asset with price $S,$ satisfies the Black-Scholes equation
  $$ \frac{\delta V}{\delta t} + \frac{1}{2}\sigma^2 S^2\frac{\delta V^2}{\delta S^2} + rS \frac{\delta V}{\delta S} - rV =0,$$
 
with suitable terminal and boundary conditions:
$$V_C(S,T) = \max \{S-K,0 \}$$
$$V_P(S,T) = \max \{K-S,0 \}.$$
$$V_C(S_{\min}, t) = 0, V_C(S_{\max}, t) = S_{\max}- Ke^{-r(T-t)}, $$
$$V_P(S_{\max}, t) = 0, V_P(S_{\min}, t) = Ke^{-r(T-t)}-S_{\min}. $$

In [ ]:
# Add current folder and QuantCourseBP folder to syspath to import modules
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

current = Path(Path().resolve())
sys.path.append(str(current))
sys.path.append(str(current.parents[1]))

from src.enums import *
from src.utils import *
from src.market_data import *
from src.pricer import *
# Make charts interactive
%matplotlib notebook

# Initialize market data
MarketData.initialize()

# Utility function

In [ ]:
"""The pricing function of European call option"""
def black_scholes_eur_call(spot, time_to_mat,sigma, strike, rate):
    d1_vec = ( np.log( spot / strike ) + ( rate + 0.5 * sigma**2 ) * time_to_mat ) / ( sigma * time_to_mat**0.5 )
    d2_vec = d1_vec - sigma * time_to_mat**0.5

    N_d1_vec = norm.cdf(d1_vec)
    N_d2_vec = norm.cdf(d2_vec)

    return N_d1_vec * spot - strike * np.exp((-1.0)*rate*time_to_mat) * N_d2_vec

# Create Test Example Contracts

In [ ]:
und = Stock.TEST_COMPANY
expiry = 2.0
strike = 0.95 * MarketData.get_spot()[und]
ls = LongShort.LONG

In [ ]:
derivative_type = [PutCallFwd.CALL, PutCallFwd.PUT]
model_types = [BSVolModel(und), FlatVolModel(und)]

contracts=[EuropeanContract(und, deriv_type, ls, strike, expiry) for deriv_type in derivative_type]
analytic_pricer = [EuropeanAnalyticPricer(contract, model_type, Params()) for contract in contracts for model_type in model_types]
analytical_fv= [pricer.calc_fair_value() for pricer in analytic_pricer]

print(f"FairValues:{analytical_fv}")

# Example with Explicit Method

* Create European Contract
* Create Market Model
* Create PDEParam
* Create EuropeanPDEPricer
* Calculate FV based on Explicit method
* Visualize the Call or Put prices in 2D 

In [ ]:
params_exp = PDEParams(method = BSPDEMethod.EXPLICIT)

pde_exp_pricer = [EuropeanPDEPricer(contract, model_type, params_exp) for contract in contracts for model_type in model_types]
pde_exp_fv= [pricer.calc_fair_value() for pricer in pde_exp_pricer]




In [ ]:
print(f"PDE Explicit method FairValues:{pde_exp_fv}")
print(f"Analytical method FairValues:{analytical_fv}")

In [ ]:
Errors = [f"{(price1-price2)/price1 *100:.4f}%" for price1, price2 in zip(pde_exp_fv, analytical_fv)]
print(f"Error between the Analytical and PDE Explicit method FairValues:{Errors}")

# Attributes needed to create visualization

In [ ]:
und_step = pde_exp_pricer[0].und_step
time_step = pde_exp_pricer[0].time_step
s_min= pde_exp_pricer[0].stock_min
s_max=pde_exp_pricer[0].stock_max
sigma = pde_exp_pricer[0].bsPDE.sigma
spot = pde_exp_pricer[0].initial_spot
rate = pde_exp_pricer[0].interest_rate
num_of_und_steps = pde_exp_pricer[0].bsPDE.num_of_und_steps

# Plot the graph for the first contract for different S values

In [ ]:
import matplotlib.pyplot as plt
sol_exp = pde_exp_pricer[0].grid[:,1]
Spots = np.linspace(s_min + und_step, s_max, num_of_und_steps+1)

%matplotlib inline
plt.plot(Spots, sol_exp, "x-", label="Explicit")
plt.plot(Spots, black_scholes_eur_call(Spots, expiry, sigma, strike, rate), label="Analytical")

plt.xlim(0, s_max)
plt.ylim(0,s_max)
plt.legend()
plt.show()

# Example with Non-Convergence with Explicit Params

In [ ]:
params_exp = PDEParams(und_step = 0.01, time_step=0.001, method = BSPDEMethod.EXPLICIT)

pde_exp_pricer = EuropeanPDEPricer(contracts[0], model_types[0], params_exp) 
pde_exp_fv= pde_exp_pricer.calc_fair_value()


In [ ]:
print(f"PDE Explicit method FairValues:{pde_exp_fv}")

# Example with Implicit Method

* Create European Contract
* Create Market Model
* Create PDEParams
* Create EuropeanPDEPricer
* Calculate FV based on Implicit method
* Visualize the Call or Put prices in 2D

In [ ]:
param_imp = PDEParams(method = BSPDEMethod.IMPLICIT)

pde_imp_pricer = [EuropeanPDEPricer(contract, model_type, param_imp) for contract in contracts for model_type in model_types]

#TODO: Implement the FairValue Calculations
# pde_imp_fv 

In [ ]:
print(f"PDE Implicit method FairValues:{pde_imp_fv}")
print(f"Analytical method FairValues:{analytical_fv}")

In [ ]:
Errors_imp = [f"{(price1-price2)/price1 *100:.4f}%" for price1, price2 in zip(pde_imp_fv,analytical_fv)]
print(f"Error between the Analytical and PDE Explicit method FairValues:{Errors_imp}")

# Compare Implicit Method with Analytical Pricer

In [ ]:
sol_imp = pde_imp_pricer[0].grid[:,1]


plt.plot(Spots, sol_imp, "x-", label="Implicit")
plt.plot(Spots, black_scholes_eur_call(Spots, expiry, sigma, strike, rate), label="Analytical")

plt.xlim(0, s_max)
plt.ylim(0,s_max)
plt.legend()
plt.show()

# Example with Crank Nicolson Method

* Create European Contract
* Create Market Model
* Create PDEParams
* Create GenericPDEPricer
* Calculate FV and Greeks based on Crank_Nicolson methods
* Visualize the Call and Put prices in 2D and 3D

In [ ]:
param_crn = PDEParams(method = BSPDEMethod.CRANK_NICOLSON)

pde_crn_pricer = [EuropeanPDEPricer(contract, model_type, param_crn) for contract in contracts for model_type in model_types]
pde_crn_fv= [pricer.calc_fair_value() for pricer in pde_crn_pricer]


In [ ]:
print(f"PDE Crank-Nicolson method FairValues:{pde_crn_fv}")
print(f"Analytical method FairValues:{analytical_fv}")

In [ ]:
Errors_crn = [f"{(price1-price2)/price1 *100:.4f}%" for price1, price2 in zip(pde_crn_fv,analytical_fv)]
print(f"Error between the Analytical and PDE Crank Nicolson method FairValues:{Errors_crn}")

In [ ]:
sol_imp = pde_crn_pricer[0].grid[:,1]

plt.plot(Spots, sol_imp, "x-", label="Crank Nicolson")
plt.plot(Spots, black_scholes_eur_call(Spots, expiry, sigma, strike, rate), label="Analytical")

plt.xlim(0, s_max)
plt.ylim(0,s_max)
plt.legend()
plt.show()

# Pricing American Option with Black Scholes PDE

<b>Properties of American Option:<b>
* The price of an American Option is atleast as much as the European Option with the same properties:
  $V_t^{am} \geq V_t^{eur}, \ for \, any\, \;  t \in [0,T]$
* An American option has at least the value of the payoff at any time $t.$
* There are different numerical techniques to price an American Option: LongStaff-Schwarz, PDE, Tree methods 
* American Call contract will never be exercised early unless the stock has a dividend.
* For a sufficient Low Price an American Put will be exercised

<b>Consider the Porfolio:<b>
* Consider an American put has a value lower than $(K-S)^+$
Strategy:
* Take Long positions in both the Asset and in the money Put Option
* Exercise the Put option immediately and sell the stock at higher price K. 
* This creates an arbitrage opportunity until the Put price becomes at least the payoff.
* Thus,
$$V_{t,c}^{am} \geq \max(S_t - K,0), \ for \, any\, \;  t \in [0,T]$$
$$V_{t,p}^{am} \geq \max(K- S_t,0), \ for \, any\, \;  t \in [0,T]$$

In [ ]:
params_exp = PDEParams(method = BSPDEMethod.EXPLICIT)
contracts=[AmericanContract(und, deriv_type, ls, strike, expiry) for deriv_type in derivative_type]

pde_exp_pricer = [AmericanPDEPricer(contract, model_type, params_exp) for contract in contracts for model_type in model_types]
pde_exp_fv= [pricer.calc_fair_value() for pricer in pde_exp_pricer]

In [ ]:
print(f"PDE Explicit method FairValues:{pde_exp_fv}")

# Conclusion

* We have seen the implemention of European/American Call/Put options pricing via PDE methods


